# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key) 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv = "output_data\cities2.csv"

csv_path= pd.read_csv(csv)

csv_path.head()



,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Tiznit Province,25,MA,1585551920,70,29.58,-9.50,49.136,0.56
1,1,Mandera,32,KE,1585551728,11,3.94,41.87,84.992,0.89
2,2,Katsuura,75,JP,1585551701,61,35.13,140.30,53.006,3.60
3,3,Yar-Sale,43,RU,1585551755,93,66.83,70.83,19.508,7.96
4,4,Fukuechō,75,JP,1585551786,76,32.69,128.84,55.400,2.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) 

locations = csv_path[['Latitude', 'Longitude']]
weights = csv_path['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=300,point_radius=5))
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Weather Criteria temp lower than 90, but higher than 70.Wind speed less than 5mph. Cloudiness under 5.

#csv_path.columns
csv_path_updated = csv_path.loc[(csv_path["Max Temp"]<= 90) & (csv_path["Max Temp"]>70) & (csv_path["Wind Speed"] < 5) & (csv_path["Cloudiness"] < 5)]

csv_path_updated = csv_path_updated.dropna()
print(csv_path_updated.head())

#new map
# locations2 = csv_path_updated[['Latitude', 'Longitude']]
# weights2 = csv_path_updated['Humidity']
# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(locations2, weights=weights2, dissipating=False, max_intensity=300,point_radius=5))
# fig


     Unnamed: 0            City  Cloudiness Country        Date  Humidity  \
31           31     Keti Bandar           0      PK  1585551926        31   
35           35          Riyadh           0      SA  1585551556        33   
80           80  Saint-Philippe           2      RE  1585551407        70   
124         124        Petatlán           0      MX  1585551944        65   
130         130            Tual           0      ID  1585551945        67   

     Latitude  Longitude  Max Temp  Wind Speed  
31      24.14      67.45    86.540        2.23  
35      24.69      46.72    71.600        4.10  
80     -21.36      55.77    89.600        3.10  
124     17.52    -101.27    76.010        2.09  
130     -5.67     132.75    84.542        1.41  


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = csv_path_updated[["City","Country","Latitude","Longitude"]]
hotel_df["Hotel Name"]=""

hotel_df.head()

C:\Users\Mikey\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,City,Country,Latitude,Longitude,Hotel Name
31,Keti Bandar,PK,24.14,67.45,
35,Riyadh,SA,24.69,46.72,
80,Saint-Philippe,RE,-21.36,55.77,
124,Petatlán,MX,17.52,-101.27,
130,Tual,ID,-5.67,132.75,


In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [13]:
for index, row in hotel_df.iterrows():
    
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    
    params["location"]=f"{Latitude},{Longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    address = requests.get(base_url,params=params)
    
    address = address.json()
    
     # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipping, Missing data....")

hotel_df

C:\Users\Mikey\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,City,Country,Latitude,Longitude,Hotel Name
31,Keti Bandar,PK,24.14,67.45,Haji laloo memon house
35,Riyadh,SA,24.69,46.72,InterContinental Riyadh
80,Saint-Philippe,RE,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
124,Petatlán,MX,17.52,-101.27,Hotel California Petatlán
130,Tual,ID,-5.67,132.75,Grand Vilia Hotel
138,Elizabeth City,US,36.29,-76.25,Hampton Inn Elizabeth City
148,Māngrol,IN,21.12,70.12,Hotel The Royal Honours
163,Chui,UY,-33.70,-53.46,Nuevo Hotel Plaza
166,Esperance,AU,-33.87,121.90,Hospitality Esperance
191,Saint-Joseph,RE,-21.37,55.62,"""Plantation Bed and Breakfast"


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
 # Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))